# Imports

In [1]:
# Import utils
import numpy as np
import pandas as pd
import copy
import time
import datetime as dt
from joblib import dump, load, Parallel, delayed
import os
from tqdm import tqdm

# Import Weights Model
from WeightsModel import PreProcessing
from WeightsModel import MaxQFeatureScaler
from WeightsModel import MaxQDemandScaler
from WeightsModel import RandomForestWeightsModel

# Import Weighted SAA Model
from WeightedSAA import WeightedSAA
from WeightedSAA import RobustWeightedSAA
from WeightedSAA import RollingHorizonOptimization

# Import Experiment
from Experiment import Experiment

# Setup the experiment

**Experiment data** consists of four data sets.

- **ID_Data** (pd.DataFrame) stores identifiers (in particular the product (SKU) identifier and the timePeriod (sale_yearweek) identifier)
- **X_Data** (pd.DataFrame) is the 'feature matrix', i.e., each row is a feature vector $x_{j,n}$ of product $j$ at time $n$
- **Y_Data** (pd.DataFrame) is the demand time series data, i.e., each row is a demand observations $d_{j,n}$ of product $j$ at time $n$
- **X_Data_Columns** (pd.DataFrame) provides 'selectors' for local vs. global feature sets

Data is loaded before each experiment using **load_data()**.

We run an experiment for all given products (SKUs) $k=1,...,M$ over a test planning horizon $t=1,...,T$ with $T=13$ for three different cost parameter settings $\{K, u, h, b\}$ that vary the critical ratio ($CR=\frac{b}{b+h}$) of holding and backlogging yielding
- $CR=0.50$: $\{K=100, u=0.5, h=1, b=1\}$
- $CR=0.75$: $\{K=100, u=0.5, h=1, b=3\}$
- $CR=0.90$: $\{K=100, u=0.5, h=1, b=9\}$

Experiments are run for different choices of the look-ahead $\tau=0,...,4$. For robust models, we vary the parameter $e=\{1,3,6,9,12\}$ (multiplier of the in-sample standard deviation of demand) defining product and sample sepcific uncertainty sets.

In [2]:
# Setup the experiment
experiment_setup = dict(

    # Set paths
    path_data = '/home/fesc/DDDInventoryControl/Data',
    path_weightsmodel = '/home/fesc/DDDInventoryControl/Data/WeightsModel',
    path_results = '/home/fesc/DDDInventoryControl/Data/Results',
    
    # Weights models
    global_weightsmodel = 'rfwm_global', 
    local_weightsmodel = 'rfwm_local', 

    # Optimization models
    GwSAA = 'GwSAA',
    GwSAAR = 'GwSAAR',
    wSAA = 'wSAA',
    wSAAR = 'wSAAR',
    SAA = 'SAA',
    ExPost = 'ExPost',
    
    # Set identifier of start period of testing horizon
    timePeriodsTestStart = 114,

    # Set product identifiers
    products = range(1,460+1),   # Products (SKUs) k=1,...,M
    
    # Set problem params
    T = 13,             # Planning horizon T
    ts = range(1,13+1), # Periods t=1,...,T of the planning horizon
    taus = [0,1,2,3,4], # Look-aheads tau=0,...,4
    es = [1,3,6,9,12],  # Uncertainty set specifications e=1,...,12
       
    # Set cost params
    cost_params = [
        {'CR': 0.50, 'K': 100, 'u': 0.5, 'h': 1, 'b': 1},
        {'CR': 0.75, 'K': 100, 'u': 0.5, 'h': 1, 'b': 3},
        {'CR': 0.90, 'K': 100, 'u': 0.5, 'h': 1, 'b': 9}
    ],
    
    # Set gurobi params
    gurobi_params = {
    
        'LogToConsole': 0, 
        'Threads': 1, 
        'NonConvex': 2, 
        'PSDTol': 1e-3, # 0.1%
        'MIPGap': 1e-3, # 0.1%
        'NumericFocus': 0, 
        'obj_improvement': 1e-3, # 0.1%
        'obj_timeout_sec': 3*60, # 3 min
        'obj_timeout_max_sec': 10*60, # 10 min
        
    },
    
    # Global weights model params
    global_weightsmodel_params = dict(
    
        # Meta parameters
        model_params = {
            'oob_score': True,
            'random_state': 12345,
            'n_jobs': 4,
            'verbose': 0
        },

        # Hyper params search grid
        hyper_params_grid = {
            'n_estimators': [500, 1000],
            'max_depth': [None],
            'min_samples_split': [x for x in range(20, 100, 20)],  
            'min_samples_leaf': [x for x in range(10, 100, 10)],  
            'max_features': [x for x in range(8, 136, 8)],   
            'max_leaf_nodes': [None],
            'min_impurity_decrease': [0.0],
            'bootstrap': [True],
            'max_samples': [0.80, 0.85, 0.90, 0.95, 1.00]
        },    

        # Tuning params
        tuning_params = {     
            'n_iter': 100,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 8,
            'verbose': 0
        },    

        # Tuning using random search
        random_search = True
    ),
    
    # Local weights model params
    local_weightsmodel_params = dict(
    
        # Meta parameters
        model_params = {
            'oob_score': True,
            'random_state': 12345,
            'n_jobs': 1,
            'verbose': 0
        },

        # Hyper params search grid
        hyper_params_grid = {
            'n_estimators': [25, 50],
            'max_depth': [None],
            'min_samples_split': [x for x in range(2, 20, 2)],  
            'min_samples_leaf': [x for x in range(2, 10, 2)],  
            'max_features': [x for x in range(4, 96, 4)],   
            'max_leaf_nodes': [None],
            'min_impurity_decrease': [0.0],
            'bootstrap': [True],
            'max_samples': [0.80, 0.85, 0.90, 0.95, 1.00]
        },    

        # Tuning params
        tuning_params = {     
            'n_iter': 100,
            'scoring': {'MSE': 'neg_mean_squared_error'},
            'return_train_score': True,
            'refit': 'MSE',
            'random_state': 12345,
            'n_jobs': 32,
            'verbose': 0
        },    

        # Tuning using random search
        random_search = True
        
    )
)

# Make all experiment variables visible locally
locals().update(experiment_setup)

# Initialize Experiment
experiment = Experiment(**experiment_setup)

# Global Training and Samping

The two global models (using 'Global Training and Sampling') are **Rolling Horizon Global Weighted SAA (GwSAA)**, which is our model, and **Rolling Horizon Global Robust Weighted SAA (GwSAA-R)**, which is the analogous model with robust extension. Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'global' distribution $\{\{w_{j,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$, with weight functions $w_{j,t,\tau}(\,\cdot\,)$ trained (once for all products) on data $S_{t,\tau}^{\,\text{Global}}=\{\{(x_{j,t}^{\,i},d_{j,t}^{\,i},...,d_{j,t+\tau}^{\,i})\}_{i=1}^{N_{j,t,\tau}}\}_{j=1}^{M}$.

We first load **global** experiment data and then preprocess the data for all look-aheads $\tau=1,...,4$ and periods $t=1,...,T$ upfront. With this, we can later easily load and reuse the data which is needed for several steps along the experiment pipeline. Preprocessing includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences and mapping corresponding features accordingly. Furthermore, features and demands are scaled for training (and later rescaled for sampling). 

The weights models - and thus the data used, weight functions, and weights per sample - are the same for the two global models **GwSAA** and **GwSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor). Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$).

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features, features_to_scale, features_to_scale_with = experiment.load_data('global')

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for global models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart, tau, T, 
                                           features, features_to_scale, features_to_scale_with, 
                                           X_scaler=MaxQFeatureScaler(q_outlier=0.975), 
                                           y_scaler=MaxQDemandScaler(q_outlier=0.975))

    # Save
    _ = dump(data, path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')      

## Tune weights model

To tune the hyper parameters of the global random forest weights model, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['global_weightsmodel_params'])

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
       
    # Using t=1 (i.e, data available before start of testing)
    t=1

    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
    
    # Extract preprocessed data
    locals().update(data[t])
    
    # Initialize
    pp = PreProcessing()
    wm = RandomForestWeightsModel(model_params)

    # Scale features and demands
    X_train = pp.scale(X_train, X_scalers, ids_products_train)
    y_train = pp.scale(y_train, y_scalers, ids_products_train)   

    # CV time series splits
    cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
    
    # CV search
    cv_results = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=True)
    wm.save_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_tau'+str(tau)+'.joblib')

## Fit weight functions and generate weights

We now fit the global random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$ and over periods $t=1,...,T$. This is done across all products at once (global training). Then, for each $\tau=0,...,4$ and over periods $t=1,...,T$, we generate for each product (SKU) $k=1,...,M$ the weights given the test feature $x_{k,t}$. This is done *jointly* across products (by using $x_{t}=(x_{1,t},...,x_{M,t})^{\top}$) for computational efficiency - the weights for each individual product can be extracted afterwards.

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:

    # Status
    print('#### Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
        
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
        
    # For each period t=1,...,T
    for t in ts:
        
        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
        
        # Extract preprocessed data
        locals().update(data[t])
            
        # Adjust look-ahead tau to account for end of horizon
        tau_ = min(tau,T-t)

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # Scale features and demands
        X_train, X_test = pp.scale(X_train, X_scalers, ids_products_train), pp.scale(X_test, X_scalers, ids_products_test)
        y_train, y_test = pp.scale(y_train, y_scalers, ids_products_train), pp.scale(y_test, y_scalers, ids_products_test)           
            
        # Load tuned weights model
        wm.load_cv_result(path=path_weightsmodel+'/'+global_weightsmodel+'_cv_tau'+str(tau_)+'.joblib')
        
        # Fit weight functions  
        st_exec, st_cpu = time.time(), time.process_time() 
        wm.fit(X_train, y_train)
        weightfunctions_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
      
        # Generate weights  
        st_exec, st_cpu = time.time(), time.process_time() 
        weights[t] = wm.apply(X_train, X_test)
        weights_times[t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}
        
    # Status
    print('...done in', dt.datetime.now().replace(microsecond=0) - start_time)    
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+global_weightsmodel+'_weightfunctions_times_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+global_weightsmodel+'_weights_times_tau'+str(tau)+'.joblib')    

# Local Training and Sampling

The two local models (using 'Local Training and Sampling') are **Rolling Horizon Local Weighted SAA (wSAA)**, and **Rolling Horizon Local Robust Weighted SAA (wSAA-R)**, which is the analogous model with robust extension. Given product $k$, period $t$, and look-ahead $\tau$, both models apply Weighted SAA over the 'local' distribution $\{w_{k,t,\tau}^{\,i}(x_{k,t}^{\,i}),(d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$, with weight functions $w_{k,t,\tau}(\,\cdot\,)$ trained on data $S_{k,t,\tau}^{\,\text{Local}}=\{(x_{k,t}^{\,i},d_{k,t}^{\,i},...,d_{k,t+\tau}^{\,i})\}_{i=1}^{N_{k,t,\tau}}$ for each product $k=1,...,M$ separately.

We first load **local** experiment data and then preprocess the data for all look-aheads $\tau=1,...,4$ and periods $t=1,...,T$ upfront. With this, we can later easily load and reuse the data which is needed for several steps along the experiment pipeline. Preprocessing includes reshaping demand time series into $(\tau+1)$-periods rolling look-ahead horizon sequences and mapping corresponding features accordingly.

The weights model - and thus the data used, weight functions, and weights per sample - are the same for the two local models **wSAA** and **wSAA-R**. First, we tune the hyper parameters of the random forest weights model for each given look-ahead $\tau$ (as for each look-ahead $\tau$ we have a different response for the multi-output random forest regressor) and for each product (SKU) $k=1,...,M$ separately. Second, we fit all weight functions (for each look-ahead $\tau=0,...,4$ and over periods $t=1,...,T$) for each product (SKU) $k=1,...,M$ separately and generate all weights (for each look-ahead $\tau=0,...,4$, over periods $t=1,...,T$, and for each product (SKU) $k=1,...,M$ separatey).

In [ ]:
# Load and unpack experiment data
X, y, ids_products, ids_timePeriods, features = experiment.load_data('local')

In [ ]:
# Initialize preprocessing module of the weights model
pp = PreProcessing()

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:  
    
    # Status
    print('#### Look-ahead tau='+str(tau)+'...')

    # Preprocess data for local models
    data = pp.preprocess_weightsmodel_data(X, y, ids_products, ids_timePeriods, timePeriodsTestStart, tau, T, products=products)

    # Save
    _ = dump(data, path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')      

## Tune weights model

To tune the hyper parameters of the local random forest weights model for each product (SKU) $k=1,...,M$, we use 3-fold rolling timeseries cross-validation on the training data and perform random search with 100 iterations over the specified hyper parameter search grid.

In [ ]:
# Make meta params for tuning global weights model available
locals().update(experiment_setup['local_weightsmodel_params'])

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')
        
    # Initialize
    cv_results = {}
    
    # For each product (SKU) k=1,...,M
    for product in products:

        # Using t=1 (i.e, data available before start of testing)
        t=1

        # Extract preprocessed data
        locals().update(data[product][t])

        # Initialize
        pp = PreProcessing()
        wm = RandomForestWeightsModel(model_params)
        
        # CV time series splits
        cv_folds = pp.split_timeseries_cv(n_splits=3, ids_timePeriods=ids_timePeriods_train)
        
        # CV search
        cv_results[product] = wm.tune(X_train, y_train, cv_folds, hyper_params_grid, tuning_params, random_search, print_status=False)

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True)

    # Save
    _ = dump(cv_results, path_weightsmodel+'/'+local_weightsmodel+'_cv_tau'+str(tau)+'.joblib')
    print('')

## Fit weight functions and generate weights

We now fit a local random forest weights model (i.e., the weight functions) for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately (local training). Then, for each $\tau=0,...,4$, period $t=1,...,T$, and product (SKU) $k=1,...,M$ separately, we generate the weights given the test feature $x_{k,t}$. This is done *separately* for each product (SKU) $k=1,...,M$.

In [ ]:
# Set meta params
model_params = {
    'n_jobs': 32,
    'verbose': 0
}

In [ ]:
# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Status
    print('Look-ahead tau='+str(tau)+'...')
    start_time = dt.datetime.now().replace(microsecond=0)
    
    # Initialize
    weights, weightfunctions_times, weights_times = {}, {}, {}
    
    # Load preprocessed data (alternatively, data can be preprocessed here) 
    data = load(path_weightsmodel+'/local_data_tau'+str(tau)+'.joblib')
    
    # For each product (SKU) k=1,...,M
    for product in products:
        
        # Initialize
        weights[product], weightfunctions_times[product], weights_times[product] = {}, {}, {}
    
        # For each period t=1,...,T
        for t in ts:

            # Extract preprocessed data
            locals().update(data[product][t])
                    
            # Adjust look-ahead tau to account for end of horizon
            tau_ = min(tau,T-t)
            
            # Initialize
            pp = PreProcessing()
            wm = RandomForestWeightsModel(model_params)

            # Load tuned weights model
            wm.load_cv_result(path=path_weightsmodel+'/'+local_weightsmodel+'_cv_tau'+str(tau_)+'.joblib', product=product)

            
            # Fit weight functions  
            st_exec, st_cpu = time.time(), time.process_time() 
            wm.fit(X_train, y_train)
            weightfunctions_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

            # Generate weights  
            st_exec, st_cpu = time.time(), time.process_time() 
            weights[product][t] = wm.apply(X_train, X_test)
            weights_times[product][t] = {'exec_time_sec': time.time()-st_exec, 'cpu_time_sec': time.process_time()-st_cpu}

        # Status
        print('Product '+str(product)+' of '+str(len(products))+' in', dt.datetime.now().replace(microsecond=0) - start_time, end='\r', flush=True) 
        
    # Save
    _ = dump(weights, path_weightsmodel+'/'+local_weightsmodel+'_weights_tau'+str(tau)+'.joblib')   
    _ = dump(weightfunctions_times, path_weightsmodel+'/'+local_weightsmodel+'_weightfunctions_times_tau'+str(tau)+'.joblib') 
    _ = dump(weights_times, path_weightsmodel+'/'+local_weightsmodel+'_weights_times_tau'+str(tau)+'.joblib')    
    print('')

# Rolling Horizon Optimization

...

## (a) Rolling Horizon Global Weighted SAA (GwSAA)

...

In [3]:
# Set number of cores
n_jobs = 64

In [4]:
# Make all experiment variables visible locally
locals().update(experiment_setup)

In [ ]:
# Set path
if not os.path.exists(path_results+'/'+GwSAA): os.mkdir(path_results+'/'+GwSAA)

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Initialize Experiment
    experiment = Experiment(global_weightsmodel, GwSAA, **experiment_setup)

    # Load preprocessed data (alternatively, data can be preprocessed here)
    data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')
    
    # Load weights
    weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 
    
    # Preprocess experiment data
    weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)
    
    # For each product (SKU) k=1,...,M
    with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
        resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
                                                                     samples=samples_[product], actuals=actuals_[product]) for product in products)

In [ ]:
run(wsaamodel=WeightedSAA(), actuals, samples=None, weights=None, epsilons=None, print_progress=False, return_results=False, **kwargs):

    """

In [ ]:
cost_params

In [ ]:
experiment.experiment_setup

In [ ]:
# # Set path
# if not os.path.exists(path_results+'/'+GwSAA): os.mkdir(path_results+'/'+GwSAA)

# tau=2
    
# # Print:
# print('Look-ahead tau='+str(tau)+'...')

# # Initialize Experiment
# experiment = Experiment(global_weightsmodel, GwSAA, **experiment_setup)

# # Load preprocessed data (alternatively, data can be preprocessed here)
# data = load(path_weightsmodel+'/global_data_tau'+str(tau)+'.joblib')

# # Load weights
# weights = load(path_weightsmodel+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

# # Preprocess experiment data
# weights_, samples_, actuals_ = experiment.preprocess_data(data, weights)

In [ ]:
# data[1]

In [ ]:
# # For each product (SKU) k=1,...,M
# with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
#     resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, product=product, wsaamodel=WeightedSAA(), weights=weights_[product], 
#                                                                  samples=samples_[product], actuals=actuals_[product]) for product in products)

In [ ]:
# # Set path
# if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# # For each look-ahead tau=0,...,4
# for tau in taus:
    
#     # Print:
#     print('Look-ahead tau='+str(tau)+'...')
    
#     # Prepare data
#     pp = PreProcessing()

#     # Load and extract preprocessed data (alternatively, data can be preprocessed here)
#     data = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_data_tau'+str(tau)+'.joblib')
    
#     # Load and extract weights
#     weights = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 
    
#     # Samples
#     samples_ = {}

#     # For products k=1,...,M
#     for product in products:

#         # Initialize
#         samples_[product] = {}

#         # For periods t=1,...,T
#         for t in ts:

#             # Get demand data, product identifiers, and fitted scalers
#             y_train = data[t]['y_train']
#             products_train = data[t]['products_train']
#             y_scalers = data[t]['y_scalers']

#             # Scale demand with fitted demand scaler per product
#             y_train_z = pp.scale(y_train, y_scalers, products_train)  

#             # Rescale demand with fitted demand scaler of the current product
#             y_train_zz = pp.rescale(y_train_z, y_scalers[product]) 

#             # Store demand samples
#             samples_[product][t] = copy.deepcopy(y_train_zz)

#     # Actuals
#     actuals_ = {}

#     # For products k=1,...,M
#     for product in products:

#         # Initialize
#         actuals_[product] = {}

#         # For periods t=1,...,T
#         for t in ts:

#             # Get demand actuals and product identifiers
#             y_test = data[t]['y_test']
#             products_test = data[t]['products_test']

#             # Store demand actuals
#             actuals_[product][t] = y_test[products_test==product].flatten()
            
#     # Weights   
#     weights_ = {}

#     # For products k=1,...,M
#     for product in products:

#         # Initialize
#         weights_[product] = {}

#         # For periods t=1,...,T
#         for t in ts:

#             # Get product identifiers
#             products_test = data[t]['products_test']

#             # Store weights
#             weights_[product][t] = weights[t][products_test==product].flatten()
    
#     # For each product (SKU) k=1,...,M
#     with tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
#         resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiment)(tau=tau, SKU=product, wsaamodel=WeightedSAA(), samples=samples_[product], 
#                                                                      weights=weights_[product], actuals=actuals_[product], 
#                                                                      **experiment_params) for product in products)

In [ ]:
#### WITH EXPERIMENT AS OWN CLASS ####

#### CODE NOT YET WORKING

## (b) Rolling Horizon Global Robust Weighted SAA (GwSAA-R)

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,

    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/'+GwSAAR+'_FINAL',
    'name_to_save_prefix': GwSAAR+'_FINAL',
    'print_progress': False,
    'return_results': False

}

n_jobs = 32

In [ ]:
# For each uncertainty set specification
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
    
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Prepare data
        pp = PreProcessing()

        # Load and extract preprocessed data (alternatively, data can be preprocessed here)
        data = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_data_tau'+str(tau)+'.joblib')

        # Load and extract weights
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

        # Samples
        samples_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            samples_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand data, product identifiers, and fitted scalers
                y_train = data[t]['y_train']
                products_train = data[t]['products_train']
                y_scalers = data[t]['y_scalers']

                # Scale demand with fitted demand scaler per product
                y_train_z = pp.scale(y_train, y_scalers, products_train)  

                # Rescale demand with fitted demand scaler of the current product
                y_train_zz = pp.rescale(y_train_z, y_scalers[product]) 

                # Store demand samples
                samples_[product][t] = copy.deepcopy(y_train_zz)

        # Actuals
        actuals_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            actuals_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand actuals and product identifiers
                y_test = data[t]['y_test']
                products_test = data[t]['products_test']

                # Store demand actuals
                actuals_[product][t] = y_test[products_test==product].flatten()

        # Weights   
        weights_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            weights_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get product identifiers
                products_test = data[t]['products_test']
                
                # Store weights
                weights_[product][t] = weights[t][products_test==product].flatten()

        # Epsilons  
        epsilons_ = {}
        
        # For products k=1,...,M
        for product in products:

            # Initialize
            epsilons_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand data, product identifiers, and fitted scalers
                y_train = data[t]['y_train']
                products_train = data[t]['products_train']
    
                # Calculate epsilon as e * in-sample standard deviation of current product's demand
                epsilons_[product][t] = e * np.std(y_train[products_train==product].flatten())

        # For each product (SKU) k=1,...,M
        with tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
            resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiment)(tau=tau, SKU=product, wsaamodel=RobustWeightedSAA(), 
                                                                         samples=samples_[product], weights=weights_[product], epsilons=epsilons_[product],
                                                                         actuals=actuals_[product], e=e, **experiment_params) for product in products)

In [ ]:
# For each uncertainty set specification
for e in [12]:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
    
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in [2,3,4]:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Prepare data
        pp = PreProcessing()

        # Load and extract preprocessed data (alternatively, data can be preprocessed here)
        data = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_data_tau'+str(tau)+'.joblib')

        # Load and extract weights
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

        # Samples
        samples_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            samples_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand data, product identifiers, and fitted scalers
                y_train = data[t]['y_train']
                products_train = data[t]['products_train']
                y_scalers = data[t]['y_scalers']

                # Scale demand with fitted demand scaler per product
                y_train_z = pp.scale(y_train, y_scalers, products_train)  

                # Rescale demand with fitted demand scaler of the current product
                y_train_zz = pp.rescale(y_train_z, y_scalers[product]) 

                # Store demand samples
                samples_[product][t] = copy.deepcopy(y_train_zz)

        # Actuals
        actuals_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            actuals_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand actuals and product identifiers
                y_test = data[t]['y_test']
                products_test = data[t]['products_test']

                # Store demand actuals
                actuals_[product][t] = y_test[products_test==product].flatten()

        # Weights   
        weights_ = {}

        # For products k=1,...,M
        for product in products:

            # Initialize
            weights_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get product identifiers
                products_test = data[t]['products_test']
                
                # Store weights
                weights_[product][t] = weights[t][products_test==product].flatten()

        # Epsilons  
        epsilons_ = {}
        
        # For products k=1,...,M
        for product in products:

            # Initialize
            epsilons_[product] = {}

            # For periods t=1,...,T
            for t in ts:

                # Get demand data, product identifiers, and fitted scalers
                y_train = data[t]['y_train']
                products_train = data[t]['products_train']
    
                # Calculate epsilon as e * in-sample standard deviation of current product's demand
                epsilons_[product][t] = e * np.std(y_train[products_train==product].flatten())

        # For each product (SKU) k=1,...,M
        with tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
            resultslog = Parallel(n_jobs=n_jobs)(delayed(run_experiment)(tau=tau, SKU=product, wsaamodel=RobustWeightedSAA(), 
                                                                         samples=samples_[product], weights=weights_[product], epsilons=epsilons_[product],
                                                                         actuals=actuals_[product], e=e, **experiment_params) for product in products)

In [ ]:
#### NEW CODE ####

In [ ]:
# For each uncertainty set specification
for e in es:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
    
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Initialize
        experiment = Experiment()

        # Load preprocessed data (alternatively, data can be preprocessed here)
        data = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_data_tau'+str(tau)+'.joblib')

        # Load weights
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+global_weightsmodel+'_weights_tau'+str(tau)+'.joblib') 

        # Preprocess experiment data
        weights_, samples_, actuals_, epsilons_ = experiment.preprocess_experiment_data(data, weights, e=e)


        # For each product (SKU) k=1,...,M
        with experiment.tqdm_joblib(tqdm(desc='Progress', total=len(products))) as progress_bar:
            resultslog = Parallel(n_jobs=n_jobs)(delayed(experiment.run)(tau=tau, SKU=product, wsaamodel=RobustWeightedSAA(), 
                                                                         weights=weights_[product], samples=samples_[product], 
                                                                         actuals=actuals_[product], epsilons=epsilons_[product],
                                                                         e=e, **experiment_params) for product in products)

## (c) Rolling Horizon Local Weighted SAA (wSAA)

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/'+wSAA+'_FINAL',
    'name_to_save': wSAA+'_FINAL',
    'print_progress': False,
    'return_results': False

}

n_jobs=32

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
    weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

    samples, actuals, weights = prep_samples_and_weights(samples, weights, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples[SKU], weights=weights[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

## (d) Rolling Horizon Local Robust Weighted SAA (wSAA-R)

...

In [ ]:
# Weights model names
weightsmodel_cv_name = 'cv_rfwm_local_not_reshaped'
weightsmodel_name = 'rfwm_local_not_reshaped'

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/wSAAR',
    'name_to_save_prefix': 'wSAAR',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# For each uncertainty set specification
for e in [1,3,6,9,12]:
    
    # Print:
    print('Uncertainty set parameter e='+str(e)+'...')
        
    # Update params
    experiment_params['name_to_save'] = experiment_params['name_to_save_prefix']+'_e'+str(e).replace('.', '')
    
    # Set path
    if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

    # For each look-ahead tau=0,...,4
    for tau in taus:

        # Print:
        print('...look-ahead tau='+str(tau)+'...')

        # Prepare data
        samples = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_samples_tau'+str(tau)+'.joblib')
        weights = joblib.load(PATH_WEIGHTSMODEL+'/'+weightsmodel_name+'_weights_tau'+str(tau)+'.joblib')

        samples, actuals, weights, epsilons = prep_samples_and_weights(samples, weights, e=e, SKUs=SKUs, ts=ts)

        # For each product (SKU) k=1,...,M
        with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
            resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=RobustWeightedSAA(), 
                                                                     samples=samples[SKU], weights=weights[SKU], epsilons=epsilons[SKU],
                                                                     actuals=actuals[SKU], e=e, **experiment_params) for SKU in SKUs)

## (e) Baseline model: Rolling Horizon Local Weighted SAA (SAA)

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/SAA',
    'name_to_save': 'SAA',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each look-ahead tau=0,...,4
for tau in taus:
    
    # Print:
    print('Look-ahead tau='+str(tau)+'...')
    
    # Prepare data
    samples = joblib.load(PATH_WEIGHTSMODEL+'/rfwm_local_samples_not_reshaped_tau'+str(tau)+'.joblib')
    
    samples, actuals = prep_samples_and_weights(samples, SKUs=SKUs, ts=ts)
    
    # For each product (SKU) k=1,...,M
    with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
        resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(tau=tau, SKU=SKU, wsaamodel=WeightedSAA(), 
                                                                 samples=samples[SKU], actuals=actuals[SKU], 
                                                                 **experiment_params) for SKU in SKUs)

## (f) Ex-post optimal model with deterministic demand

...

In [ ]:
# Define experiment paramaters
experiment_params = {
            
    # Cost param settings
    'cost_params': cost_params,
    
    # Gurobi meta params
    'LogToConsole': 0, 
    'Threads': 1, 
    'NonConvex': 2, 
    'PSDTol': 1e-3, # 0.1%
    'MIPGap': 1e-3, # 0.1%
    'NumericFocus': 0, 
    'obj_improvement': 1e-3, # 0.1%
    'obj_timeout_sec': 3*60, # 3 min
    'obj_timeout_max_sec': 10*60, # 10 min

    # Program meta params
    'path_to_save': PATH_RESULTS+'/ExPost',
    'name_to_save': 'ExPost',
    'print_progress': False,
    'return_results': False

}

In [ ]:
# Prepare data
samples = joblib.load(PATH_WEIGHTSMODEL+'/rfwm_local_samples_not_reshaped_tau'+str(0)+'.joblib')
actuals = {}
for SKU in SKUs:
    d = []
    for t in ts:
        d = d + [samples[SKU][t]['y_test'].item()]
    actuals[SKU] = np.array(d).reshape(1,len(d))

In [ ]:
# Set path
if not os.path.exists(experiment_params['path_to_save']): os.mkdir(experiment_params['path_to_save'])

# For each product (SKU) k=1,...,M
with tqdm_joblib(tqdm(desc='Progress', total=len(SKUs))) as progress_bar:
    resultslog = Parallel(n_jobs=32)(delayed(run_experiment)(SKU=SKU, wsaamodel=WeightedSAA(), actuals=actuals[SKU], **experiment_params) for SKU in SKUs)